In [1]:
from grand import lig_samplers as ls
from grand import lig_utils as lu

import openmm
import openmm.unit
import openff.toolkit
import pdbfixer
import parmed

import numpy as np
import mdtraj as md
from scipy.spatial import Voronoi, Delaunay
import os

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
datadir = '../../examples/kras/'
# Load protein and ligand
prot = openmm.app.PDBFile(f'{datadir}/input.pdb')
lig = openff.toolkit.Molecule(f'{datadir}/7R0M_7r0m_pose_0.sdf')
lig.generate_conformers(n_conformers=1000)
lig_top = lig.to_topology().to_openmm()
lig_pos = lig.conformers[0].to_openmm()

# Load forcefield
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml',f'{datadir}/lig_openff.xml')

In [3]:
# Add one ligand to the system
sys_top, sys_pos, ghosts = lu.add_ghosts(prot.topology, prot.positions, lig_top, lig_pos, n=1, output=f'{datadir}/ncmc_init.pdb')

100%|██████████| 1/1 [00:00<00:00, 38.82it/s]


In [4]:
# Load pdb with protein and ligand
pdb = openmm.app.PDBFile(f'{datadir}/ncmc_init.pdb')

In [5]:
frag_res_ids = []
frag_atom_ids = []
alchemical_atom_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)
        for atom in residue.atoms():
            frag_atom_ids.append(atom.index)
            alchemical_atom_ids.append(atom.index)

In [6]:
data = """
-21.2652    5.1471  -68.8848 C   0  0  0  0  0  0  0  0  0  0  0  0
-25.1684    5.7819  -73.4243 N   0  0  0  0  0  0  0  0  0  0  0  0
-22.4642    4.9718  -69.8029 C   0  0  0  0  0  0  0  0  0  0  0  0
-24.0862    6.3421  -72.8592 N   0  0  0  0  0  0  0  0  0  0  0  0
-22.6352    5.8953  -70.8378 C   0  0  0  0  0  0  0  0  0  0  0  0
-23.6760    5.6646  -71.7344 C   0  0  0  0  0  0  0  0  0  0  0  0
-24.5600    4.5713  -71.6101 C   0  0  0  0  0  0  0  0  0  0  0  0
-24.3949    3.6785  -70.5414 C   0  0  0  0  0  0  0  0  0  0  0  0
-23.3410    3.8858  -69.6483 C   0  0  0  0  0  0  0  0  0  0  0  0
-25.4500    4.7159  -72.7192 C   0  0  0  0  0  0  0  0  0  0  0  0
-23.1259    2.7429  -68.3680 Cl  0  0  0  0  0  0  0  0  0  0  0  0
-23.6648    7.1895  -73.2428 H   0  0  0  0  0  0  0  0  0  0  0  0
-20.5104    5.8068  -69.3142 H   0  0  0  0  0  0  0  0  0  0  0  0
-21.5771    5.5410  -67.9222 H   0  0  0  0  0  0  0  0  0  0  0  0
-20.7795    4.1868  -68.7160 H   0  0  0  0  0  0  0  0  0  0  0  0
-21.9800    6.7415  -70.9547 H   0  0  0  0  0  0  0  0  0  0  0  0
-25.0565    2.8342  -70.4187 H   0  0  0  0  0  0  0  0  0  0  0  0
-26.2759    4.0753  -72.9909 H   0  0  0  0  0  0  0  0  0  0  0  0
"""

# Convert the multi-line string to a list of lists of floats
lines = data.strip().split('\n')
array_data = [list(map(float, line.split()[:3])) for line in lines]

# Convert the list of lists to a NumPy array
lig_coord = np.array(array_data)


In [7]:
#lig_coord = []
lig_mass = []
for residue in lig_top.residues():
    for atom in residue.atoms():
        #lig_coord.append(lig_pos[int(atom.index)].value_in_unit(openmm.unit.nanometer))
        lig_mass.append(atom.element._mass)

ref_coords = parmed.geometry.center_of_mass(np.array(lig_coord), np.array(lig_mass))
print(ref_coords)

[-23.566379608615314 4.600636484380879 -70.61910695454638]


In [8]:
ref_coords = ref_coords/10

In [9]:
pdb_system = ff.createSystem(pdb.topology, nonbondedMethod=openmm.app.NoCutoff)
dcd_reporter = openmm.app.DCDReporter(f"{datadir}/ncmc_kras.dcd", reportInterval=50)
state_reporter = openmm.app.StateDataReporter(f"{datadir}/ncmc_kras_state.csv",step=True, time=True, potentialEnergy=True, kineticEnergy=True,
                                              totalEnergy=True, temperature=True, volume=True, density=True, reportInterval=50)

In [10]:
# Initialize Sampler
sampler = ls.BaseNCMCSampler(pdb_system,pdb.topology,pdb.positions,reporters=[dcd_reporter,state_reporter],alchemical_atoms=alchemical_atom_ids, overwrite=True,
                             nsteps_neq=3000,nsteps_eq=1000, frag_atoms=frag_atom_ids, insert_points_list=[openmm.Vec3(-2.3566379608615314, 0.4600636484380879, -7.061910695454638)]
                            )

In [11]:
# Run sampler
sampler.ncmc_move(10)

  0%|          | 0/10 [00:00<?, ?it/s]

2.5147705078125 kJ/mol 19.069015502929688 kJ/mol 625.0296249389648 kJ/mol -234.8232421875 kJ/mol rejected


 10%|█         | 1/10 [00:04<00:40,  4.47s/it]

2.34014892578125 kJ/mol 1.9526214599609375 kJ/mol 685.5507507324219 kJ/mol 246.767578125 kJ/mol rejected


 20%|██        | 2/10 [00:09<00:36,  4.53s/it]

...

 80%|████████  | 8/10 [00:36<00:09,  4.53s/it]

121.13168334960938 kJ/mol -17.123626708984375 kJ/mol 626.8941650390625 kJ/mol 449.6767578125 kJ/mol rejected


 90%|█████████ | 9/10 [00:40<00:04,  4.51s/it]

125.3260498046875 kJ/mol 80.48408508300781 kJ/mol 744.3294830322266 kJ/mol 61.162109375 kJ/mol rejected


100%|██████████| 10/10 [00:45<00:00,  4.50s/it]
